<h2><b>Automate & document iMOD runs</b></h2>

This script works as a wrapper for the iMOD runs performed for the WaterScape project.<br>
It'll read WS_RunLog.xlsx, identify queued runs, then execute the corresponding snakemake file(s).

# 0. Prep

## 0.0. Libraries

In [1]:
import WS_Mdl as WS
import os
import pandas as pd
import subprocess as sp

## 0.1. Paths

In [2]:
path_RunLog = os.path.join(WS.path_WS, 'Mng/WS_RunLog.xlsx')

# 1. Load RunLog and read inputs

In [3]:
DF = pd.read_excel(path_RunLog, sheet_name='RunLog').dropna(subset='runN')

In [4]:
DF_q = DF.loc[ (DF['Start Status'] == 'Queued') & ((DF['End Status'].isna()) | (DF['End Status']=='Failed')) ] # _q for queued

In [45]:
!snakemake -p -s $path_Smk --cores 10

Assuming unrestricted shared filesystem usage.
host: UU120516
Building DAG of jobs...
Provided cores: 10
Rules claiming more threads will be scaled down.
Job stats:
job             count
------------  -------
all                 1
log_Init            1
log_SP_dates        1
test1               1
total               4

Select jobs to execute...
Execute 2 jobs...

[Thu May  1 16:38:10 2025]
localrule test1:
    output: test
    jobid: 3
    reason: Missing output files: test
    resources: tmpdir=C:\Users\Karam014\AppData\Local\Temp
Shell command: None

[Thu May  1 16:38:10 2025]
localrule log_Init:
    output: C:/OD/WS_Mdl/models/NBr/code/snakemake/temp1
    jobid: 2
    reason: Missing output files: C:/OD/WS_Mdl/models/NBr/code/snakemake/temp1
    resources: tmpdir=C:\Users\Karam014\AppData\Local\Temp
Shell command: None
[Thu May  1 16:38:10 2025]
Finished jobid: 3 (Rule: test1)
1 of 4 steps (25%) done
RuleException:
PermissionError in file "C:\OD\WS_Mdl\models\NBr\code\snakemake\NBr11

In [44]:
os.remove(r'C:\OD\WS_Mdl\models\NBr\test\.snakemake_timestamp')
os.rmdir(r'C:\OD\WS_Mdl\models\NBr\test', )
os.remove(r'C:\OD\WS_Mdl\models\NBr\code\snakemake\temp2')

In [ ]:
for i, Se_Ln in DF_q.iterrows():
    path_Smk = os.path.join(WS.path_WS, f'models/{Se_Ln["model alias"]}/code/snakemake/{Se_Ln["MdlN"]}.smk')
    model_name = Se_Ln["MdlN"]

    print(f"\n🟦 Running model {model_name}...")
    print(f"   Snakefile: {path_Smk}\n" + "-"*60)

    try:
        result = sp.run(['snakemake', '-s', '-p', path_Smk], check=True, capture_output=True, text=True)
        print(result.stdout)
        print(f"✅ Model {model_name} finished successfully.\n" + "="*60)
    except sp.CalledProcessError as e:
        print(f"--- Snakemake stdout ---\n{e.stdout}\n")
        print(f"--- Snakemake stderr ---\n{e.stderr}\n")
        print(f"❌ Model {model_name} failed!\n" + "="*60)



🟦 Running model NBr11...
   Snakefile: C:/OD/WS_Mdl\models/NBr/code/snakemake/NBr11.smk
------------------------------------------------------------
--- Snakemake stdout ---


--- Snakemake stderr ---
SyntaxError in file "C:\OD\WS_Mdl\models\NBr\code\snakemake\NBr11.smk", line 16:
invalid syntax:
        [temp("code/snakemake/temp2")]



❌ Model NBr11 failed!


In [ ]:
stop

# Junkyard

stop

d_Cmds.items()

## Run models in parallel using a process pool
with Pool(num_cores) as pool:
    pool.starmap(WS.run_Mdl, [(row, DF_Opt) for _, row in queued_Sims.iterrows()])

d_Cmds

for i, Se_Ln in DF.loc[DF['Status']=='Queued'].iterrows():
    MdlN = Se_Ln['MdlN']
    print(f"--- Executing RunMng for {MdlN}")

    # Get default directories
    d_paths = WS.get_MdlN_paths(MdlN)
    MdlN_B, path_Mdl, path_MdlN, path_INI, path_BAT, path_PRJ = (d_paths[k] for k in ['MdlN_B', 'path_Mdl', 'path_MdlN', "path_INI_S", "path_BAT_S", "path_PRJ_S"])
        
    # Define commands and their working directories
    d_Cmds = {path_BAT: os.path.dirname(path_BAT),
              "activate WS": os.path.dirname(path_BAT),
              f"WS_Mdl add_OBS {MdlN} {DF_Opt.loc[DF_Opt['MdlN']==MdlN, 'add_OBS'].values[0]}": os.path.dirname(path_BAT),
              r'.\RUN.BAT': path_MdlN}

    # Generate log file path
    log_file = os.path.join(path_MdlN, f'Tmnl_Out_{MdlN}.txt')
    os.makedirs(os.path.dirname(log_file), exist_ok=True)

    with open(log_file, 'w', encoding='utf-8') as f:
        for Cmd, cwd in d_Cmds.items():
            try:
                print(f" -- Executing: {Cmd}")

                # Run command in blocking mode, capturing output live
                process = sp.run(Cmd, shell=True, cwd=cwd,
                                 capture_output=True, text=True, check=True)

                # Write output to file
                f.write(f'{"*"*50}  START OF COMMAND   {"*"*50}\n')
                f.write(f"Command: {Cmd}\n{"-"*120}\n\n")
                f.write(f"Output:\n{process.stdout}\n{"-"*120}\n\n")
                f.write(f"Errors:\n{process.stderr}\n{"-"*120}\n\n")
                f.write(f"Return Code: {process.returncode}\n{"-"*120}\n\n")
                f.write(f'{"*"*50}  END OF COMMAND     {"*"*50}\n\n')

                print(f"  - ✓")

            except sp.CalledProcessError as e:
                print(f"  - ❌: {Cmd}\nError: {e.stderr}")
                f.write(f"ERROR: {e.stderr}\n")


os.path.basename(log_file)

s = 'NBr1'
Mdl, SimN = "".join(filter(str.isalpha, s)), "".join(filter(str.isdigit, s))
Mdl, SimN

## Examples

## Change the color of a specific cell (e.g., A1)
fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")  # Yellow color
sheet['A1'].fill = fill

sheet['F5'] = 'Development'
sheet['F6'] = 'development'
sheet['F7'] = 'Development_'

## Save the modified workbook
wb.save('modified_file.xlsx')